In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.signal import cwt, find_peaks_cwt, ricker, welch
from scipy.stats import linregress
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import lightgbm as lgb
import gc

pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings('ignore')

In [2]:
sensors = pd.read_csv('input/sensors.csv')

In [3]:
tags = pd.read_csv('input/tags.csv')

In [4]:
target_coke = pd.read_csv('input/coke_target.csv')

In [5]:
# функция для определения к какой установке относится фича
def get_tool_group(x):
    if 'Установка ректификации' in x:
        return 'Установка ректификации'
    elif 'Установка дегидрирования' in x:
        return 'Установка дегидрирования'
    elif 'Установка компримирования' in x:
        return 'Установка компримирования'

In [6]:
#все расходы газа в кг/ч а тут кПа, это странно (расход в единицах давления?). заменим на кг/ч
tags.loc[tags.feature=='f_16','units'] = 'кг/ч'

In [7]:
tags['tool_group'] = tags['description'].apply(get_tool_group)

In [8]:
# фичи почти целиком из пропусков, по ним не группируем
tags = tags[~tags.feature.isin(['f0','f1','f2','f41','f42'])]

In [9]:
# получаем группы фичей по установке+единицам измерения
tool_measure_groups = tags.groupby(['tool_group','units'])['feature'].apply(lambda x: ', '.join(x)).reset_index()

In [10]:
# считаем среднее значение по группам фич установка+единицы измерения
for features in tool_measure_groups['feature']:
    features_list = features.split(', ')
    if len(features_list) > 1:
        sensors[features+'_mean'] = sensors[features_list].mean(axis=1)

In [11]:
# удаляем фичи в которых больше 50% пропущенных значений.
for col in sensors.columns:
    if (sensors[col].isnull().sum() / sensors.shape[0]) > 0.5:
        #print(col + ' dropped because of NaNs (NaN share is: {})'.format(sensors[col].isnull().sum() / sensors.shape[0]))
        sensors.drop(col, axis=1, inplace=True)

f_0 dropped because of NaNs (NaN share is: 0.9559975889089813)
f_1 dropped because of NaNs (NaN share is: 0.9566003616636528)
f_2 dropped because of NaNs (NaN share is: 0.956298975286317)
f_36 dropped because of NaNs (NaN share is: 0.5853676913803496)
f_37 dropped because of NaNs (NaN share is: 0.5853676913803496)
f_38 dropped because of NaNs (NaN share is: 0.5852169981916817)
f_39 dropped because of NaNs (NaN share is: 0.5846895720313442)
f_40 dropped because of NaNs (NaN share is: 0.5747438215792646)
f_41 dropped because of NaNs (NaN share is: 0.9613471971066908)
f_42 dropped because of NaNs (NaN share is: 0.961045810729355)
f_41, f_42_mean dropped because of NaNs (NaN share is: 0.9609704641350211)
f_36, f_37, f_38, f_39_mean dropped because of NaNs (NaN share is: 0.5846142254370102)


In [13]:
#делаем несколько групп фичей по "смыслу"
# содержание компонентов в этане (в названии переменной есть опечатка)
comp_in_metan_group = ['f_3','f_4','f_5', 'f_6']

# расход этана
etan_spend_group = ['f_8', 'f_9', 'f_10']

# расход рециркулирующего газа
gas_spend_group = ['f_11','f_12', 'f_13']

# расход продувочного газа
gas_spend_reactors_group = ['f_14', 'f_15', 'f_16', 'f_17', 'f_18',
                            'f_19', 'f_22', 'f_23', 'f_24', 'f_26', 'f_27']

# температуры потоков
temp_group = ['f_20', 'f_28', 'f_30', 'f_31']

# средние температуры
avg_temp_group = ['f_21', 'f_25', 'f_29']

# расход сернистого вещества
S_spend = ['f_33', 'f_34', 'f_35']

In [14]:
feature_groups = [comp_in_metan_group,
                  etan_spend_group,
                  gas_spend_group,
                  gas_spend_reactors_group,
                  temp_group,
                  avg_temp_group,
                  S_spend]

In [15]:
# по каждой группе фичей считаем среднее, стандартное отклонение, размах и сумму
for group_name in feature_groups:
    sensors[str(group_name) + '_mean'] = sensors[group_name].mean(axis=1)
    sensors[str(group_name) + '_std'] = sensors[group_name].std(axis=1)
    sensors[str(group_name) + '_range'] = sensors[group_name].max(axis=1) - sensors[group_name].min(axis=1)
    sensors[str(group_name) + '_sum'] = sensors[group_name].sum(axis=1)

In [16]:
# еще фичи. разницы температур между этапами.
# разница температур между R2 и R1
sensors['temp_d_1'] = sensors['f_20'] - sensors['f_30']

# разница температур между R3 и R2
sensors['temp_d_2'] = sensors['f_31'] - sensors['f_20']

# разница температур между R4 и R3
sensors['temp_d_3'] = sensors['f_28'] - sensors['f_31']

# разница средних температур между печами H3 и Н2
sensors['temp_d_owen_1'] = sensors['f_25'] - sensors['f_21']

# разница средних температур между печами H4 и Н3
sensors['temp_d_owen_2'] = sensors['f_29'] - sensors['f_25']

# разница температур на выходе из печи H2 и входе в реактор R2
sensors['owen_to_reactor1'] = sensors['f_20'] - sensors['f_21']

# разница температур на выходе из печи H3 и входе в реактор R3
sensors['owen_to_reactor2'] = sensors['f_25'] - sensors['f_31']

# разница температур на выходе из печи H4 и входе в реактор R4
sensors['owen_to_reactor3'] = sensors['f_29'] - sensors['f_28']

In [17]:
# фичи из библиотеки tsfresh https://github.com/blue-yonder/tsfresh
# взял просто функции из кода библиотеки, чуточку подправил фичу с линрегом
def abs_energy(x):
    """
    Returns the absolute energy of the time series which is the sum over the squared values
    .. math::
        E = \\sum_{i=1,\ldots, n} x_i^2
    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.dot(x, x)


def mean_abs_change(x):
    """
    Returns the mean over the absolute differences between subsequent time series values which is
    .. math::
        \\frac{1}{n} \\sum_{i=1,\ldots, n-1} | x_{i+1} - x_{i}|
    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    return np.mean(np.abs(np.diff(x)))


def skewness(x):
    """
    Returns the sample skewness of x (calculated with the adjusted Fisher-Pearson standardized
    moment coefficient G1).
    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    return pd.Series.skew(x)


def kurtosis(x):
    """
    Returns the kurtosis of x (calculated with the adjusted Fisher-Pearson standardized
    moment coefficient G2).
    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    return pd.Series.kurtosis(x)


def linear_trend(x, param='slope'):
    """
    Calculate a linear least-squares regression for the values of the time series versus the sequence from 0 to
    length of the time series minus one.
    This feature assumes the signal to be uniformly sampled. It will not use the time stamps to fit the model.
    The parameters control which of the characteristics are returned.
    Possible extracted attributes are "pvalue", "rvalue", "intercept", "slope", "stderr", see the documentation of
    linregress for more information.
    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :param param: contains dictionaries {"attr": x} with x an string, the attribute name of the regression model
    :type param: list
    :return: the different feature values
    :return type: pandas.Series
    """
    # todo: we could use the index of the DataFrame here

    linReg = linregress(range(len(x)), x)
    return linReg[0]
    #return [("attr_\"{}\"".format(config["attr"]), getattr(linReg, config["attr"]))
    #        for config in param]

In [18]:
# функция для получения номера недели месяца
from math import ceil

def week_of_month(dt):
    """ 
    Returns the week of the month for the specified date.
    """
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

In [20]:
sensors['timestamp'] = pd.to_datetime(sensors['timestamp'])

In [21]:
sensors['day'] = sensors['timestamp'].dt.day

In [22]:
sensors['month'] = sensors['timestamp'].dt.month

In [23]:
# генерим кучу фичей
# фичи отбирались руками
# путем удаления-добавления групп фичей(либо по применяемой функции либо по размеру окна)
for col in sensors.columns:
    if col not in  ['timestamp', 'day', 'month', 'week_of_month']:
        # todo inner loop
        # относительные разности текущих значений с различными лагами
        sensors[str(col) + '_lag1'] = (sensors[col] - sensors[col].shift(1)) / sensors[col]
        sensors[str(col) + '_lag3'] = (sensors[col] - sensors[col].shift(3)) / sensors[col]
        sensors[str(col) + '_lag6'] = (sensors[col] - sensors[col].shift(6)) / sensors[col]
        sensors[str(col) + '_lag12'] = (sensors[col] - sensors[col].shift(12)) / sensors[col]
        sensors[str(col) + '_lag24'] = (sensors[col] - sensors[col].shift(24)) / sensors[col]
        
        # относительная разность текущего значения с средним по окну за предыдущий период
        sensors[str(col)+'_rolling6_diff'] = (sensors[col] - pd.rolling_mean(sensors[col], window=6, min_periods=1))/\
                                                pd.rolling_mean(sensors[col], window=6, min_periods=1)
        sensors[str(col)+'_rolling3_diff'] = (sensors[col] - pd.rolling_mean(sensors[col], window=3, min_periods=1))/\
                                                pd.rolling_mean(sensors[col], window=3, min_periods=1)
        sensors[str(col)+'_rolling12_diff'] = (sensors[col] - pd.rolling_mean(sensors[col], window=12, min_periods=1))/\
                                                pd.rolling_mean(sensors[col], window=12, min_periods=1)
        sensors[str(col)+'_rolling24_diff'] = (sensors[col] - pd.rolling_mean(sensors[col], window=24, min_periods=1))/\
                                                pd.rolling_mean(sensors[col], window=24, min_periods=1)

        # скользящее отклонение
        sensors[str(col)+'_rolling6_std'] = pd.rolling_std(sensors[col], window=6, min_periods=1)
        sensors[str(col)+'_rolling3_std'] = pd.rolling_std(sensors[col], window=3, min_periods=1)
        sensors[str(col)+'_rolling12_std'] = pd.rolling_std(sensors[col], window=12, min_periods=1)
        sensors[str(col)+'_rolling24_std'] = pd.rolling_std(sensors[col], window=24, min_periods=1)

        # окном применяем по сути minmax скейлинг, чтобы понять где находится текущее значение относительно предыдущего периода
        sensors[str(col)+'_rolling24_range_scale'] = (sensors[col] - pd.rolling_min(sensors[col], window=24, min_periods=1))/\
        (pd.rolling_max(sensors[col], window=24, min_periods=1) - pd.rolling_min(sensors[col], window=24, min_periods=1))
        
        sensors[str(col)+'_rolling12_range_scale'] = (sensors[col] - pd.rolling_min(sensors[col], window=12, min_periods=1))/\
        (pd.rolling_max(sensors[col], window=12, min_periods=1) - pd.rolling_min(sensors[col], window=12, min_periods=1))
        
        sensors[str(col)+'_rolling6_range_scale'] = (sensors[col] - pd.rolling_min(sensors[col], window=6, min_periods=1))/\
        (pd.rolling_max(sensors[col], window=6, min_periods=1) - pd.rolling_min(sensors[col], window=6, min_periods=1))
        
        # окном применяем tsfresh фичи
        # "энергия сигнала" - скалярное произведение сигнала саомго на себя
        # коэффициент линейной регрессии по окну (куда движется сигнал) - тренд
        # средний модуль изменений внутри окна (насколько сильно скачет сигнал в среднем в окне)
        sensors[str(col)+'_rolling24_energy'] = pd.rolling_apply(sensors[col], window=24, func=abs_energy, min_periods=1)
        sensors[str(col)+'_rolling24_abschange'] = pd.rolling_apply(sensors[col], window=24, func=mean_abs_change, min_periods=1)
        sensors[str(col)+'_rolling24_linear_trend'] = pd.rolling_apply(sensors[col], window=24, func=linear_trend, min_periods=1)
        sensors[str(col)+'_rolling48_energy'] = pd.rolling_apply(sensors[col], window=48, func=abs_energy, min_periods=1)
        sensors[str(col)+'_rolling12_energy'] = pd.rolling_apply(sensors[col], window=12, func=abs_energy, min_periods=1)


Wall time: 15min 24s


In [24]:
# бьем на трейн/тест
train = sensors.iloc[:target_coke.shape[0],:]
test = sensors.iloc[target_coke.shape[0]:,:]

In [25]:
y = target_coke['target']

In [26]:
# удаляем таймстепм
train.drop('timestamp', axis=1, inplace=True)
test.drop('timestamp', axis=1, inplace=True)

In [27]:
# обучаем 15 лгбм'ов с разными сидами, сохраняем результат для дальнейшего бленда
# фичей много, поэтому модель сильно регуляризуем:
# уменьшаем число листьев, коэф. регуляризации побольше, саб и колсемплы поменьше.
# эти параметры просто подобрал руками в пару итераций, оказались достаточно хороши чтобы их и оставить
np.random.seed(1337)
predictions = pd.DataFrame()
for i in range(15):
    lgbm_model=lgb.LGBMRegressor(
                                  boosting_type= 'gbdt',
                                  objective='regression',
                                  colsample_bytree= 0.5,
                                  learning_rate= 0.01,
                                  n_estimators= 2000,
                                  num_leaves= 16,
                                  metric= 'rmse',
                                  subsample= 0.5,
                                  reg_lambda=10,
                                  seed= np.random.randint(1,500)
                                )
    lgbm_model.fit(train, y)
    prediction = lgbm_model.predict(test)
    predictions[str(i) + '_pred'] = prediction
    #print('predicted: ' + str(i))

predicted: 0
predicted: 1
predicted: 2
predicted: 3
predicted: 4
predicted: 5
predicted: 6
predicted: 7
predicted: 8
predicted: 9
predicted: 10
predicted: 11
predicted: 12
predicted: 13
predicted: 14


In [28]:
predictions.to_csv('coke_blend/v6 (best public) 15lgbm.csv', index=False)